# Парсер новостных пабликов вк

In [1]:
import datetime
import pickle
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time 

from tqdm import tqdm   # https://github.com/tqdm/tqdm

Выясняем сколько я уже скачал (id поста со стены для остановки докачки)

In [2]:
news_publics = {               # posts   subscribers
    "rbc" : "25232578",        # 47761    700k
    "ria" : "15755094",        # 32061    2500k
    'tass' : "26284064",       # 128930   620k
    'interfax' : "24228287",   # 48687    36k
    
    'vedomosty' : "15548215",  # 65056    680k
    'komersant' : "23482909",  # 125477   260k
    'lenta' : "67991642",      # 98294    502k
    'rg' : "23304496",         # 166240   273k
    'lentach_1' : "29534144",  # 19491    2000k
    'lentach_2' : "125004421", # 14400    426k
    'meduza' : "76982440",     # 42924    600k
    'rt' : "40316705"          # 53414    1200k
}

In [3]:
my_user_id = 91857120
version = '5.73' 

with open('secret_token.txt') as f:
    token = f.read()

In [4]:
def vkDownload(method, parameters, token=token, version=version):
    url = f'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    response = requests.get(url).json()    
    return response

def makeBatch(uids, size=25):
    batches = [uids[i:i + size] for i in range(0,len(uids),size)]
    return batches 

vkDownload('users.get','user_ids=ppilif')

{'response': [{'first_name': 'Филипп',
   'id': 6045249,
   'last_name': 'Ульянкин'}]}

# 1. Group posts

In [6]:
def getPosts(group_id, already_done=set()):
    """
        Download the content of all posts that are in the group
    """
    
    news = [ ]

    n = vkDownload('wall.get','owner_id=-{}'.format(group_id))['response']['count']
    print(f"There are {n} posts in the group {group_id}")
    
    already_done_bool = False

    for i in range(0, n + 100, 100):
        time.sleep(0.3)

        wall = vkDownload('wall.get','owner_id=-{}&count=100&offset='.format(group_id)+str(i))['response']['items']

        for i,item in enumerate(wall):
            cur_news = { }
            if cur_news.get('views'):
                cur_news['views'] = item['views']['count']
            cur_news['likes'] = item['likes']['count']
            cur_news['reposts'] = item['reposts']['count']
            cur_news['comments'] = item['comments']['count']
            cur_news['text'] = item['text']
            cur_news['date'] = item['date']

            cur_news['from_id'] = item['from_id']
            cur_news['id'] = item['id']
            cur_news['owner_id'] = item['owner_id']
            
            if item['id'] in already_done:
                already_done_bool = True

            if item.get('attachments'):
                for jtem in item['attachments']:
                    if jtem['type'] == 'link':
                        ktem = jtem['link']
                        cur_news['url'] = ktem['url']
                        cur_news['title'] = ktem['title']
            news.append(cur_news) 
            
        if already_done_bool:
            break
            
    return news

In [7]:
# test
# group_id = news_publics['rbc']
# news = getPosts(group_id)

In [ ]:
len(news)

In [ ]:
news[0]

# 2. Comments 

In [7]:
def prepareComments(group_id, posts):
    """
        We will download 25 requests at a time, this function prepares requests
        for subsequent use, understanding how many comments under which post,
        throwing out posts with zero comments and making threes (group, post, offset)
    """

    triple_comments = [ ] # group_id, post_id, offset)

    # create batches
    posts_batch = makeBatch(posts)

    for batch in tqdm(posts_batch):
        time.sleep(0.4)

        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(group_id) + '","post_id":"' + str(bt) + '"}),'

        requests.get(begin + middle[:-1] + end)

        s = requests.get(begin + middle[:-1] + end)

        # prepare requests
        for post,item in zip(batch, s.json()['response']):
            if item is False:
                print(f'False in the post {post} of the group {group_id}')
                continue

            n = item['count']
            if n == 0:
                continue 
            else:
                for i in range(0, n, 100):
                    triple_comments.append((group_id, post, i))
    return triple_comments


def getComments(triple_comments):
    
    comments = [ ] 
    precom_batch = makeBatch(triple_comments, 15)

    for batch in tqdm(precom_batch):
        time.sleep(0.4)
        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(bt[0]) + '","post_id":"' + str(bt[1]) + '",\
            "need_likes":"1","preview_length":"0","count":"100","offset":"'+ str(bt[2])+'"}),'

        s = requests.get(begin + middle[:-1] + end)
        
        cur_com = [ ]
        for item, post in zip(s.json()['response'], batch):
            for jtem in item['items']:
                jtem['likes'] = jtem['likes']['count']
                jtem['post_id'] = post[1]
                jtem['group_id'] = post[0]
                cur_com.append(jtem)
                
        comments.extend(cur_com) 
        
    return comments    

In [ ]:
for name in list(news_publics.keys()):
    
    group_id = news_publics[name]
    news = getPosts(group_id)

    with open('news_data/{}_vk_posts'.format(name), 'wb') as f:
        pickle.dump(news, f)

    posts_id = [new['id'] for new in news]

    print(f'For {name} we need {len(posts_id)} posts')
    req = prepareComments(group_id, posts_id)

    print(f'Number of requests: {len(req)}')

    comments = getComments(req)

    setlen = len(set([str(comment['post_id']) + '_' + str(comment['id']) for comment in comments]))
    print(f'Number of comments: {len(comments)}, uniq comments: {setlen}')

    for comment in comments[-100:]:
        print(comment['post_id'],comment['id'])
        print(comment['text'])
        print('--------------------')

    with open('news_data/{}_vk_comments'.format(name), 'wb') as f:
        pickle.dump(comments, f)


    print('====================================================================')

# 3. Download additional comments

In [8]:
news_publics = {               # posts  subscribers
    "rbc" : "25232578",        # 47761    700k
    "ria" : "15755094",        # 32061    2500k
    'tass' : "26284064",       # 128930   620k
    'interfax' : "24228287",   # 48687    36k
    
    'vedomosty' : "15548215",  # 65056    680k
    'komersant' : "23482909",  # 125477   260k
    'lenta' : "67991642",      # 98294    502k
    'rg' : "23304496",         # 166240   273k
    'lentach_1' : "29534144",  # 19491    2000k
    'lentach_2' : "125004421", # 14400    426k
    'meduza' : "76982440",     # 42924    600k
    'rt' : "40316705"          # 53414    1200k
}

In [10]:
list(news_publics.keys())

['rbc',
 'ria',
 'tass',
 'interfax',
 'vedomosty',
 'komersant',
 'lenta',
 'rg',
 'lentach_1',
 'lentach_2',
 'meduza',
 'rt']

In [27]:
list(news_publics.keys())[8:]

['lentach_1', 'lentach_2', 'meduza', 'rt']

In [25]:
for name in list(news_publics.keys())[8:]:
    
    print(name)
    
    data_path = f"news_data/vk_news/posts/{name}_vk_posts"

    with open(data_path, 'rb') as f:
        cur_news = pickle.load(f)

    already_downloaded = set([post['id'] for post in cur_news])
    len(already_downloaded)
    

    # качаем посты с группы и сохраняем файл с ними 
    group_id = news_publics[name]
    news = getPosts(group_id, already_done=already_downloaded)
    
    
    print('Old news:', len(cur_news))
    cur_news.extend(news)
    print('Total news:', len(cur_news))

    with open(data_path + '_v2', 'wb') as f:
        pickle.dump(cur_news, f)
        
    posts_id = [new['id'] for new in news]

    print('For {} we need {} posts'.format(name,len(posts_id)))
    req = prepareComments(group_id, posts_id)

    print('Number of requests: {}'.format(len(req)))

    comments = getComments(req)

    setlen = len(set([str(comment['post_id']) + '_' + str(comment['id']) for comment in comments]))
    print(f'Number of comments: {len(comments)}, uniq comments: {setlen}')

    for comment in comments[-100:]:
        print(comment['post_id'],comment['id'])
        print(comment['text'])
        print('--------------------')
              
    data_path = f"news_data/vk_news/comments/{name}_vk_comments_2020-06-01_sen"

    with open(data_path, 'rb') as f:
        cur_comments = pickle.load(f)

    print('Old comments:', len(cur_comments))
    cur_comments.extend(comments)
    print('Total comments:', len(cur_comments))
    
    with open(f"news_data/vk_news/{name}_vk_comments_2020-11-01", 'wb') as f:
        pickle.dump(cur_comments, f)
        
    print('===========================================================================')

vedomosty
В группе 15548215 должно быть 86550 постов
старых новостей: 81995
всего новостей: 86695


  0%|          | 0/188 [00:00<?, ?it/s]

Для vedomosty нужно скачать 4700 постов


  0%|          | 0/287 [00:00<?, ?it/s]

Число запросов: 4304


...

старых комментов: 39837
всего новостей: 103815
lentach_1
В группе 29534144 должно быть 48202 постов


  0%|          | 0/136 [00:00<?, ?it/s]

старых новостей: 30515
всего новостей: 33915
Для lentach_1 нужно скачать 3400 постов


 10%|▉         | 13/136 [02:47<26:26, 12.90s/it]


KeyboardInterrupt: 